# Crewai + Personalised Educational Assistant

#### Refered documentation: 
#### https://docs.crewai.com/en/guides/crews/first-crew
#### https://docs.crewai.com/en/learn/create-custom-tools
#### https://docs.crewai.com/en/tools/ai-ml/codeinterpretertool#code-interpreter
#### https://docs.crewai.com/en/learn/llm-selection-guide#manager-llm
#### https://docs.crewai.com/en/learn/custom-manager-agent#custom-manager-agent

In [13]:
import os
from typing import List, Optional

from pydantic import BaseModel
from crewai import Agent, Task, Crew, Process
from crewai_tools import CodeInterpreterTool

In [15]:
os.environ["OPENAI_API_KEY"] = "<API_KEY>"

In [17]:
class CodeIssue(BaseModel):
    type: str  
    message: str
    line: Optional[int] = None
    suggestion: Optional[str] = None


class CodeAnalysisResult(BaseModel):
    issues: List[CodeIssue]
    overall_summary: str


class CodeCorrectionResult(BaseModel):
    corrected_code: str
    notes: Optional[str] = None

In [19]:
code_interpreter = CodeInterpreterTool()

In [21]:
code_analyzer = Agent(
    role="Code Analyzer",
    goal=(
        "Analyze the given Python code and identify syntax, runtime, and logical issues. "
        "Use the CodeInterpreterTool to execute or probe the code when needed."
    ),
    backstory=(
        "You are a seasoned Python debugging expert. You understand stack traces, "
        "edge cases, recursion bugs, and subtle logic errors. You are very explicit "
        "about what is wrong and how to fix it."
    ),
    tools=[code_interpreter],
    verbose=True,
    allow_delegation=False,
)

code_corrector = Agent(
    role="Code Corrector",
    goal=(
        "Take the analysis findings and produce a corrected, clean, runnable version "
        "of the Python code while preserving the original intent."
    ),
    backstory=(
        "You are a senior Python engineer. You write clear, idiomatic, and tested code. "
        "You fix bugs without changing the high-level purpose of the function or script."
    ),
    verbose=True,
    allow_delegation=False,
)

manager = Agent(
    role="Code Review Manager",
    goal=(
        "Oversee the entire debugging workflow. Ensure the analyzer finds real issues, "
        "the corrector fixes them properly, and the overall process is coherent."
    ),
    backstory=(
        "You are a technical lead who coordinates code review efforts. "
        "You don't write code yourself in this flow; you manage, validate, and steer."
    ),
    allow_delegation=True,
    verbose=True,
)

In [23]:
analysis_task = Task(
    description=(
        "You are the Code Analyzer.\n\n"
        "Analyze the following Python code for errors:\n\n"
        "```python\n"
        "{code}\n"
        "```\n\n"
        "You MUST:\n"
        "1. Consider potential syntax issues, runtime errors, and logical bugs.\n"
        "2. When helpful, call the 'Code Interpreter' tool (CodeInterpreterTool) by "
        "   passing the entire code string via the 'code' argument and optionally "
        "   a 'libraries_used' list.\n"
        "   - Use it to execute or probe the code and observe exceptions or output.\n"
        "3. Summarize each issue you find with:\n"
        "   - type: 'syntax' | 'runtime' | 'logical' | 'style' | 'other'\n"
        "   - message: human-readable explanation\n"
        "   - line: line number if you can infer it (otherwise null)\n"
        "   - suggestion: how to fix or improve it\n\n"
        "Return ONLY valid JSON matching the CodeAnalysisResult schema:\n"
        "{\n"
        '  \"issues\": [\n'
        "    {\n"
        '      \"type\": \"logical\",\n'
        '      \"message\": \"Description of the problem\",\n'
        '      \"line\": 10,\n'
        '      \"suggestion\": \"Suggested fix.\"\n'
        "    }\n"
        "  ],\n"
        '  \"overall_summary\": \"High-level summary of the issues.\"\n'
        "}\n\n"
        "Do NOT wrap the JSON in code fences. No extra commentary."
    ),
    agent=code_analyzer,
    expected_output="A JSON object matching CodeAnalysisResult: {issues: [...], overall_summary: str}.",
    output_json=CodeAnalysisResult,
)

correction_task = Task(
    description=(
        "You are the Code Corrector.\n\n"
        "You will receive the original code and the analyzer's report as context "
        "(the manager will ensure you see them above in the conversation).\n\n"
        "Your job:\n"
        "- Fix ALL identified issues (syntax, runtime, logical).\n"
        "- Keep the original goal of the code intact (e.g., if it's a Fibonacci "
        "  function, it should still compute Fibonacci correctly).\n"
        "- Make the code clear and Pythonic.\n"
        "- You may use the analysis content (issues list, suggestions) to guide you.\n\n"
        "Return ONLY valid JSON matching CodeCorrectionResult:\n"
        "{\n"
        '  \"corrected_code\": \"\"\"<fully corrected Python code>\"\"\",\n'
        '  \"notes\": \"Optional explanation of the changes you made.\"\n'
        "}\n\n"
        "Do NOT wrap the JSON in code fences. No extra commentary."
    ),
    agent=code_corrector,
    expected_output="A JSON object matching CodeCorrectionResult: {corrected_code: str, notes: str}.",
    output_json=CodeCorrectionResult,
)

In [25]:
debug_crew = Crew(
    agents=[code_analyzer, code_corrector],
    tasks=[analysis_task, correction_task],
    manager_agent=manager,                 
    process=Process.hierarchical,  
    planning=True,                  
    verbose=True,
)

In [27]:
def run_debugger(code: str):
    result = debug_crew.kickoff(inputs={"code": code})
    return result

In [29]:
def print_debugger_result(result):
    """
    Pretty-print analysis + correction results from the crew output.
    """
    print("\n" + "=" * 80)
    print("🔍 CODE ANALYSIS RESULT")
    print("=" * 80)

    analysis = result.tasks_output[0].json_dict
    issues = analysis.get("issues", [])
    summary = analysis.get("overall_summary", "")

    print("\nOverall summary:")
    print(f"- {summary}")

    print("\nIssues:")
    if not issues:
        print("  - No issues reported.")
    else:
        for i, issue in enumerate(issues, 1):
            print(f"\nIssue {i}:")
            print(f"  Type      : {issue.get('type')}")
            print(f"  Message   : {issue.get('message')}")
            if issue.get("line") is not None:
                print(f"  Line      : {issue.get('line')}")
            if issue.get("suggestion"):
                print(f"  Suggestion: {issue.get('suggestion')}")

    print("\n" + "=" * 80)
    print("🛠️  CODE CORRECTION RESULT")
    print("=" * 80)

    correction = result.tasks_output[1].json_dict
    corrected_code = correction.get("corrected_code", "")
    notes = correction.get("notes", "")

    print("\nCorrected Code:\n")
    print(corrected_code)

    if notes:
        print("\nNotes from Code Corrector:")
        print(f"- {notes}")

    print("\n" + "=" * 80)
    print("✅ MANAGER OVERSIGHT (HIERARCHICAL PROCESS)")
    print("=" * 80)
    print(
        "\nNote: In the hierarchical process, the manager agent has overseen "
        "task delegation and validation implicitly. Its reasoning is present "
        "in the crew traces / logs, even though we didn't define a separate "
        "manager-only task."
    )
    print("\n" + "=" * 80)

## Example 1

In [31]:
buggy_code = """
def fib(n):
if n <= 1:
    return 1
return fib(n-1) + fib(n-3)

print(fib(5))
"""

result = run_debugger(buggy_code)
print_debugger_result(result)

╭──────────────────────────────────────────── Crew Execution Started ─────────────────────────────────────────────╮
│                                                                                                                 │
│  Crew Execution Started                                                                                         │
│  Name: crew                                                                                                     │
│  ID: a6fe0cf9-9a7e-4ce0-9f9f-ffc0b4339e5e                                                                       │
│  Tool Args:                                                                                                     │
│                                                                                                                 │
│                                                                                                                 │
╰─────────────────────────────────────────────────────────────────────────────────────────────────────────────────╯

Output()


[2025-12-06 15:48:55][INFO]: Planning the crew execution


╭──────────────────────────────────────────────── Task Completion ────────────────────────────────────────────────╮
│                                                                                                                 │
│  Task Completed                                                                                                 │
│  Name: 6b51966c-3bf5-4639-ab8b-94de18cb60d8                                                                     │
│  Agent: Task Execution Planner                                                                                  │
│  Tool Args:                                                                                                     │
│                                                                                                                 │
│                                                                                                                 │
╰─────────────────────────────────────────────────────────────────────────────────────────────────────────────────╯

Output()

...

Output()

Output()

╭───────────────────────────────────────────── ✅ Agent Final Answer ─────────────────────────────────────────────╮
│                                                                                                                 │
│  Agent: Code Review Manager                                                                                     │
│                                                                                                                 │
│  Final Answer:                                                                                                  │
│  {                                                                                                              │
│    "corrected_code": "def fib(n):\n    if n <= 1:\n        return n\n    return fib(n-1) + fib(n-2)",           │
│    "notes": "Corrected the indentation of the 'if' statement and return lines to properly nest inside the fib   │
│  function. Also fixed the Fibonacci logic by replacing fib(n-3) with fib(n-2) to correctly compute the          │
│  Fibonacci sequence."                                                                                           │
│  }                                                                                                              │
│                                                                                                                 │
╰─────────────────────────────────────────────────────────────────────────────────────────────────────────────────╯


🔍 CODE ANALYSIS RESULT

Overall summary:
- The code has an indentation syntax error preventing it from running, and a logical error in the Fibonacci function's recursive sum parameters. Fixing indentation and adjusting the Fibonacci calculation will resolve all errors.

Issues:

Issue 1:
  Type      : syntax
  Message   : Indentation error: the 'if' statement and subsequent code inside the 'fib' function are not indented properly.
  Line      : 2
  Suggestion: Indent the 'if' statement and the return statements inside the 'fib' function properly to be within the function block.

Issue 2:
  Type      : logical
  Message   : Incorrect Fibonacci logic: the function sums fib(n-1) and fib(n-3) instead of the proper fib(n-1) and fib(n-2).
  Line      : 4
  Suggestion: Change the return statement to 'return fib(n-1) + fib(n-2)' to correctly calculate the Fibonacci sequence.

🛠️  CODE CORRECTION RESULT

Corrected Code:

def fib(n):
    if n <= 1:
        return n
    return fib(n-1) + fib(n-2

╭──────────────────────────────────────────────── Task Completion ────────────────────────────────────────────────╮
│                                                                                                                 │
│  Task Completed                                                                                                 │
│  Name: 698c8547-064c-4800-a7c1-b8db103a7cc5                                                                     │
│  Agent: Code Review Manager                                                                                     │
│  Tool Args:                                                                                                     │
│                                                                                                                 │
│                                                                                                                 │
╰─────────────────────────────────────────────────────────────────────────────────────────────────────────────────╯

In [33]:
def fib(n):
    if n <= 1:
        return n
    return fib(n-1) + fib(n-2)

In [35]:
fib(10)

55

## Example 2

In [43]:
buggy_code = """
import statistics

def normalize_scores(scores):
    # supposed to normalize between 0 and 1
    max_score = max(scores)
    min_score = min(scores)
    norm = []
    for s in scores:
        norm.append(s / max_score - min_score)
    return norm

def compute_statistics(scores):
    avg = sum(scores) / len(scores)
    med = statistics.mode(scores) 
    top = scores[0]    
    return avg, med, top

def detect_failing_students(scores, passing_threshold=0.4):
    failing = []
    for i in range(len(scores)):
        if scores[i] < passing_threshold:
            failing.append(scores[i]) 
        elif scores[i] == None:
            failing.append(i) 
    return failing

def analyze_grades(raw_scores):
    normalized = normalize_scores(raw_scores)
    avg, med, top = compute_statistics(raw_scores)
    fails = detect_failing_students(normalized, passing_threshold=0.5)

    print("Average:", avg)
    print("Median:", med)
    print("Top Score:", top)
    print("Failing students:", fails)

scores = [82, 90, 76, 90, 58, 0, 90, 45, 82]

analyze_grades(scores)
"""

result2 = run_debugger(buggy_code)

╭──────────────────────────────────────────── Crew Execution Started ─────────────────────────────────────────────╮
│                                                                                                                 │
│  Crew Execution Started                                                                                         │
│  Name: crew                                                                                                     │
│  ID: a6fe0cf9-9a7e-4ce0-9f9f-ffc0b4339e5e                                                                       │
│  Tool Args:                                                                                                     │
│                                                                                                                 │
│                                                                                                                 │
╰─────────────────────────────────────────────────────────────────────────────────────────────────────────────────╯

Output()


[2025-12-06 15:54:45][INFO]: Planning the crew execution


╭──────────────────────────────────────────────── Task Completion ────────────────────────────────────────────────╮
│                                                                                                                 │
│  Task Completed                                                                                                 │
│  Name: cf495d99-fb34-4d78-896e-19f41cbf7951                                                                     │
│  Agent: Task Execution Planner                                                                                  │
│  Tool Args:                                                                                                     │
│                                                                                                                 │
│                                                                                                                 │
╰─────────────────────────────────────────────────────────────────────────────────────────────────────────────────╯

╭─────────────────────────────────────────────── 🤖 Agent Started ────────────────────────────────────────────────╮
│                                                                                                                 │
│  Agent: Code Review Manager                                                                                     │
│                                                                                                                 │
│  Task: You are the Code Analyzer.                                                                               │
│                                                                                                                 │
│  Analyze the following Python code for errors:                                                                  │
│                                                                                                                 │
│  ```python                                                                                                      │
│                                                                                                                 │
│  import statistics                                                                                              │
│                                                                                                                 │
│  def normalize_scores(scores):                                                                                  │
│      # supposed to normalize between 0 and 1                                                                    │
│      max_score = max(scores)                                                                                    │
│      min_score = min(scores)                                                                                    │
│      norm = []                                                                                                  │
│      for s in scores:                                                                                           │
│          norm.append(s / max_score - min_score)                                                                 │
│      return norm                                                                                                │
│                                                                                                                 │
│  def compute_statistics(scores):                                                                                │
│      avg = sum(scores) / len(scores)                                                                            │
│      med = statistics.mode(scores)                                                                              │
│      top = scores[0]                                                                                            │
│      return avg, med, top                                                                                       │
│                                                                                                                 │
│  def detect_failing_students(scores, passing_threshold=0.4):                                                    │
│      failing = []                                                                                               │
│      for i in range(len(scores)):                                                                               │
│          if scores[i] < passing_threshold:                                                                      │
│              failing.append(scores[i])                                                                          │
│          elif scores[i] == None:                                                                                │
│              failing.append(i)                                                                                  │
│      return failing                                    

...

╭──────────────────────────────────────────────── Task Completion ────────────────────────────────────────────────╮
│                                                                                                                 │
│  Task Completed                                                                                                 │
│  Name: 45682926-8dca-4d8b-ab3e-389a3b9438e6                                                                     │
│  Agent: Code Review Manager                                                                                     │
│  Tool Args:                                                                                                     │
│                                                                                                                 │
│                                                                                                                 │
╰─────────────────────────────────────────────────────────────────────────────────────────────────────────────────╯

╭─────────────────────────────────────────────── 🤖 Agent Started ────────────────────────────────────────────────╮
│                                                                                                                 │
│  Agent: Code Review Manager                                                                                     │
│                                                                                                                 │
│  Task: You are the Code Corrector.                                                                              │
│                                                                                                                 │
│  You will receive the original code and the analyzer's report as context (the manager will ensure you see them  │
│  above in the conversation).                                                                                    │
│                                                                                                                 │
│  Your job:                                                                                                      │
│  - Fix ALL identified issues (syntax, runtime, logical).                                                        │
│  - Keep the original goal of the code intact (e.g., if it's a Fibonacci   function, it should still compute     │
│  Fibonacci correctly).                                                                                          │
│  - Make the code clear and Pythonic.                                                                            │
│  - You may use the analysis content (issues list, suggestions) to guide you.                                    │
│                                                                                                                 │
│  Return ONLY valid JSON matching CodeCorrectionResult:                                                          │
│  {                                                                                                              │
│    "corrected_code": """<fully corrected Python code>""",                                                       │
│    "notes": "Optional explanation of the changes you made."                                                     │
│  }                                                                                                              │
│                                                                                                                 │
│  Do NOT wrap the JSON in code fences. No extra commentary.1. Receive the original code and the Code Analyzer's  │
│  report containing identified issues. 2. Carefully review each reported issue, understanding its context and    │
│  implications. 3. Start making necessary corrections in the 'normalize_scores' function by properly             │
│  normalizing the scores to fall within the range [0, 1]. 4. Fix the 'compute_statistics' function to handle     │
│  cases where scores may not have a unique mode. 5. Modify 'detect_failing_students' to only check for scores    │
│  less than the passing threshold without unnecessarily checking for 'None'. 6. Ensure the overall structure     │
│  and functionality of the original code are preserved. 7. Conduct thorough testing of the corrected code to     │
│  confirm that all issues are resolved and that the code operates as intended. 8. Return the corrected code      │
│  with any notes regarding the changes made, following the specified JSON schema.                                │
│                                                                                                                 │
╰─────────────────────────────────────────────────────────────────────────────────────────────────────────────────╯

Output()

╭───────────────────────────────────────────── ✅ Agent Final Answer ─────────────────────────────────────────────╮
│                                                                                                                 │
│  Agent: Code Review Manager                                                                                     │
│                                                                                                                 │
│  Final Answer:                                                                                                  │
│  {                                                                                                              │
│    "corrected_code": "import statistics\n\ndef normalize_scores(scores):\n    if not scores:\n        return    │
│  []\n    min_score = min(scores)\n    max_score = max(scores)\n    if max_score == min_score:\n        return   │
│  [0.0 for _ in scores]\n    return [(s - min_score) / (max_score - min_score) for s in scores]\n\n\ndef         │
│  compute_statistics(scores):\n    if not scores:\n        return None, None, None, None\n    avg =              │
│  statistics.mean(scores)\n    median = statistics.median(scores)\n    top = max(scores)\n    low =              │
│  min(scores)\n    return avg, median, top, low\n\n\ndef detect_failing_students(scores, passing_threshold):\n   │
│  failing_students = []\n    for i, score in enumerate(scores):\n        if score < passing_threshold:\n         │
│  failing_students.append(i)  # Collect indices of failing students\n    return failing_students\n",             │
│    "notes": "Fixed normalization formula to correctly scale scores between 0 and 1. Changed compute_statistics  │
│  to use statistics.median for median calculation and to find top score using max(scores). Updated               │
│  detect_failing_students to remove improper None check and to consistently return indices of failing            │
│  students."                                                                                                     │
│  }                                                                                                              │
│                                                                                                                 │
╰─────────────────────────────────────────────────────────────────────────────────────────────────────────────────╯

Output()

In [45]:
print_debugger_result(result2)


🔍 CODE ANALYSIS RESULT

Overall summary:
- The code contains logical errors mainly in the normalization formula, misuse of statistics functions for median calculation, incorrect identification of the top score, and inconsistent handling of failing students. These need to be fixed for the code to work as intended.

Issues:

Issue 1:
  Type      : logical
  Message   : Normalization formula is incorrect. The expression s / max_score - min_score does not scale the scores between 0 and 1 as intended.
  Line      : 10
  Suggestion: Use the correct formula (s - min_score) / (max_score - min_score) to normalize scores between 0 and 1.

Issue 2:
  Type      : logical
  Message   : The compute_statistics function uses statistics.mode to find median, which is incorrect. Also, statistics.mode can raise an exception if there are multiple modes.
  Line      : 15
  Suggestion: Replace statistics.mode with statistics.median to compute median correctly. Additionally, consider handling potential excep

In [49]:
import statistics

def normalize_scores(scores):
    if not scores:
        return []
    min_score = min(scores)
    max_score = max(scores)
    if max_score == min_score:
        return [0.0 for _ in scores]
    return [(s - min_score) / (max_score - min_score) for s in scores]


def compute_statistics(scores):
    if not scores:
        return None, None, None, None
    avg = statistics.mean(scores)
    median = statistics.median(scores)
    top = max(scores)
    low = min(scores)
    return avg, median, top, low


def detect_failing_students(scores, passing_threshold):
    failing_students = []
    for i, score in enumerate(scores):
        if score < passing_threshold:
            failing_students.append(i) 
    return failing_students

def analyze_grades(raw_scores):
    normalized = normalize_scores(raw_scores)
    avg, med, top, low = compute_statistics(raw_scores) 
    fails = detect_failing_students(normalized, passing_threshold=0.5)

    print("Average:", avg)
    print("Median:", med)
    print("Top Score:", top)
    print("Failing students:", fails)

scores = [82, 90, 76, 90, 58, 0, 90, 45, 82]

analyze_grades(scores)

Average: 68.11111111111111
Median: 82
Top Score: 90
Failing students: [5]
